# Pipeline Suitability Analysis

This notebook demonstrates the workflow for finding optimal pipeline paths using geospatial suitability analysis. The analysis consists of:

1. Loading raster factors and reference paths
2. Creating cost maps with weighted factors
3. Finding least cost paths between points
4. Comparing predicted paths with reference paths
5. Optimizing weights to improve path prediction

## Setup and Import Libraries

In [1]:
# Import required libraries
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import time
import warnings
warnings.filterwarnings('ignore')

# Import our custom pipeline analysis modules
from pipefit.data_loader import DataLoader, load_reference_path
from pipefit.cost_map import CostMapGenerator
from pipefit.path_finder import LeastCostPathFinder
from pipefit.path_evaluator import PathEvaluator

## 1. Load Dataset

First, we'll load a dataset containing raster factors and reference path.

In [ ]:
# Set the path to your dataset folder
dataset_path = "path/to/your/dataset"

# Define a factor mapping (optional)
factor_mapping = {
    'elev.tif': 'elevation',        # Map filename to standardized name
    'protected.tif': 'protected_areas',
    'cities.tif': 'proximity_to_cities',
    'slope.tif': 'slope',
    'soil.tif': 'soil_type'
}

# Create a data loader and load the data
data_loader = DataLoader(dataset_path)
data_loader.load_data(factor_mapping=factor_mapping)

# Load reference path separately
reference_path_dir = os.path.join(dataset_path, 'reference')
reference_path = load_reference_path(reference_path_dir)

# Check loaded factors
print(f"Loaded factors: {list(data_loader.factors.keys())}")
print(f"Reference path loaded: {reference_path is not None}")

## 2. Visualize Input Factors

Let's examine the normalized raster factors that we'll use to create the cost map.

In [ ]:
# Create a cost map generator
cost_generator = CostMapGenerator(data_loader.factors)

# Visualize all input factors
cost_generator.visualize_factors(figsize=(15, 10))

## 3. Generate Cost Map

Now we'll create a cost map by combining factors with weights. The weights reflect the relative importance of each factor in determining pipeline suitability.

In [ ]:
# Define weights for each factor
# These can be different from the factor names in the data files
weights = {
    'elevation': 0.3,        # Higher weight for elevation
    'protected_areas': 0.2,  # Medium weight for protected areas
    'proximity_to_cities': 0.2,  # Medium weight for city proximity
    'slope': 0.15,           # Lower weight for slope
    'soil_type': 0.15        # Lower weight for soil type
}

# Optional: Define a weight-to-factor mapping if your weight keys don't match factor names
weight_mapping = None  # We're using standardized names, so no mapping needed

# Generate cost map with default weight for any missing factors
cost_map = cost_generator.generate_cost_map(
    weights=weights,
    default_weight=0.05,  # Any factors without explicit weights get this value
    weight_mapping=weight_mapping
)

# Visualize cost map
cost_generator.visualize_cost_map(cost_map, title="Weighted Cost Map")

# Save cost map (optional)
cost_map_path = os.path.join(dataset_path, 'results', 'cost_map.tif')
os.makedirs(os.path.dirname(cost_map_path), exist_ok=True)
data_loader.save_raster(cost_map, cost_map_path)

## 4. Find Least Cost Path

Using the cost map, we'll find the least cost path between start and end points.

In [ ]:
# Define start and end points
# Option 1: Define using geographical coordinates
start_x, start_y = -120.5, 38.2  # Example coordinates (replace with actual coordinates)
end_x, end_y = -119.8, 38.7     # Example coordinates (replace with actual coordinates)

# Convert to raster indices
start_indices = data_loader.coordinates_to_indices(start_x, start_y)
end_indices = data_loader.coordinates_to_indices(end_x, end_y)

# Option 2: Define directly as raster indices
# start_indices = (100, 50)   # Example indices (row, col) (replace with actual indices)
# end_indices = (400, 300)    # Example indices (row, col) (replace with actual indices)

# Option 3: Use default start/end points (lower left to upper right)
# start_indices = None  # Will use lower left corner
# end_indices = None    # Will use upper right corner

print(f"Start point: {start_indices}, End point: {end_indices}")

# Create path finder (no need to pass transform/crs at initialization)
path_finder = LeastCostPathFinder(cost_map)

# Get metadata for later use
metadata = data_loader.get_common_metadata()

# Find least cost path using A* algorithm (default)
path_indices, path_cost = path_finder.find_path(
    start_indices, 
    end_indices, 
    method='astar',  # 'astar' or 'dijkstra'
    fully_connected=True  # Use 8-connected neighbors
)
print(f"Path found with total cost: {path_cost}")

# Visualize path on cost map
path_finder.visualize_path(path_indices, start_indices, end_indices)

# Convert path to GeoDataFrame (pass transform and crs here)
path_gdf = path_finder.path_to_geodataframe(
    path_indices, 
    path_cost, 
    transform=metadata['transform'], 
    crs=metadata['crs']
)

# Save path (optional)
path_output = os.path.join(dataset_path, 'results', 'least_cost_path.shp')
path_gdf.to_file(path_output)

## 5. Compare with Reference Path

Now we'll compare our predicted path with the reference (ground truth) path and calculate similarity metrics.

In [ ]:
# Create path evaluator with reference path passed explicitly
evaluator = PathEvaluator(
    predicted_path=path_indices,
    reference_path=reference_path,  # Pass the reference path directly
    transform=metadata['transform'],
    crs=metadata['crs']
)

# Calculate all metrics
metrics = evaluator.evaluate_all_metrics(buffer_distance=100)  # Adjust buffer distance as needed

# Display metrics
print("Path Comparison Metrics:")
for metric_name, value in metrics.items():
    print(f"{metric_name}: {value:.4f}")

# Visualize paths with metrics
evaluator.visualize_paths(title="Predicted vs Reference Path", buffer_distance=100)

## 6. Experiment with Different Weights

Let's experiment with different weight combinations to see which produces paths that better match the reference path.

In [ ]:
# Define several weight combinations to try
weight_combinations = [
    {
        'elevation': 0.5,
        'protected_areas': 0.2,
        'proximity_to_cities': 0.1,
        'slope': 0.1,
        'soil_type': 0.1
    },
    {
        'elevation': 0.2,
        'protected_areas': 0.4,
        'proximity_to_cities': 0.2,
        'slope': 0.1,
        'soil_type': 0.1
    },
    {
        'elevation': 0.2,
        'protected_areas': 0.2,
        'proximity_to_cities': 0.4,
        'slope': 0.1,
        'soil_type': 0.1
    }
]

# Store results
results = []

# Try each weight combination
for i, weights in enumerate(weight_combinations):
    print(f"\nTrying weight combination {i+1}:")
    print(weights)
    
    # Generate cost map
    cost_map = cost_generator.generate_cost_map(
        weights=weights,
        default_weight=0.05  # Use a default weight for any missing factors
    )
    
    # Create path finder
    path_finder = LeastCostPathFinder(cost_map)
    
    # Find least cost path using A* algorithm
    path_indices, path_cost = path_finder.find_path(
        start_indices, 
        end_indices, 
        method='astar'
    )
    
    # Evaluate path
    evaluator = PathEvaluator(
        predicted_path=path_indices,
        reference_path=reference_path,  # Pass reference path directly
        transform=metadata['transform'],
        crs=metadata['crs']
    )
    
    metrics = evaluator.evaluate_all_metrics(buffer_distance=100)
    
    # Store results
    results.append({
        'weights': weights,
        'path_cost': path_cost,
        'metrics': metrics
    })
    
    # Display metrics
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value:.4f}")
    
    # Visualize comparison
    evaluator.visualize_paths(title=f"Path Comparison for Weight Combination {i+1}", 
                             buffer_distance=100)

## 7. Identify Best Weight Combination

Now we'll identify which weight combination produced the best match to the reference path.

In [ ]:
# Find the best combination based on the combined score
combined_scores = [result['metrics']['combined_score'] for result in results]
best_index = combined_scores.index(min(combined_scores))
best_weights = results[best_index]['weights']

print("\nBest weight combination:")
for factor, weight in best_weights.items():
    print(f"{factor}: {weight:.2f}")

print("\nMetrics for best combination:")
for metric_name, value in results[best_index]['metrics'].items():
    print(f"{metric_name}: {value:.4f}")

## 8. Save Final Results

Finally, let's save the best cost map and path.

In [ ]:
# Generate final cost map with best weights
final_cost_map = cost_generator.generate_cost_map(
    weights=best_weights,
    default_weight=0.05
)

# Find final path using A* algorithm
path_finder = LeastCostPathFinder(final_cost_map)
final_path_indices, final_path_cost = path_finder.find_path(
    start_indices, 
    end_indices, 
    method='astar'
)

# Convert to GeoDataFrame
final_path_gdf = path_finder.path_to_geodataframe(
    final_path_indices, 
    final_path_cost, 
    transform=metadata['transform'], 
    crs=metadata['crs']
)

# Save results
results_dir = os.path.join(dataset_path, 'results')
os.makedirs(results_dir, exist_ok=True)

# Save cost map
final_cost_map_path = os.path.join(results_dir, 'final_cost_map.tif')
data_loader.save_raster(final_cost_map, final_cost_map_path)

# Save path
final_path_output = os.path.join(results_dir, 'final_least_cost_path.shp')
final_path_gdf.to_file(final_path_output)

print(f"Saved final cost map to: {final_cost_map_path}")
print(f"Saved final path to: {final_path_output}")

# Final visualization
evaluator = PathEvaluator(
    predicted_path=final_path_indices,
    reference_path=reference_path,  # Pass reference path directly
    transform=metadata['transform'],
    crs=metadata['crs']
)
evaluator.visualize_paths(title="Final Optimized Path vs Reference Path", buffer_distance=100)

## 9. Automated Weight Optimization

Let's use our optimization tools to automatically find the best weights for the factors.

### 9.1 Simple Gradient Optimization

In [ ]:
# Import the optimizer
from pipeline.weight_optimizer import SimpleGradientOptimizer

# Define the factor names to optimize
factor_names = list(data_loader.factors.keys())
print(f"Factors to optimize: {factor_names}")

# Set up datasets for optimization
# For this example, we'll use a single dataset, but you can add more
dataset_paths = [dataset_path]

# Initial weights (starting point for optimization)
initial_weights = {factor: 1.0/len(factor_names) for factor in factor_names}
print(f"Initial weights: {initial_weights}")

# Create the optimizer
optimizer = SimpleGradientOptimizer(
    datasets=dataset_paths,
    factors=factor_names,
    initial_weights=initial_weights,
    learning_rate=0.05,
    regularization=0.01
)

# Run the optimization process
best_weights, scores_history = optimizer.optimize(
    num_iterations=50,
    early_stopping_patience=10,
    verbose=True
)

# Plot the optimization history
optimizer.plot_optimization_history(scores_history)

# Save the optimized weights
weights_output = os.path.join(dataset_path, 'results', 'optimized_weights.json')
optimizer.save_weights(weights_output)

print("\nOptimized weights:")
for factor, weight in best_weights.items():
    print(f"{factor}: {weight:.4f}")

# Generate final cost map with optimized weights
final_cost_map = cost_generator.generate_cost_map(best_weights)

# Create a new path with optimized weights
path_finder = LeastCostPathFinder(final_cost_map)
optimized_path_indices, optimized_path_cost = path_finder.find_path(start_indices, end_indices)

# Evaluate the optimized path
optimized_evaluator = PathEvaluator(
    predicted_path=optimized_path_indices,
    reference_path=reference_path,
    transform=metadata['transform'],
    crs=metadata['crs']
)

optimized_metrics = optimized_evaluator.evaluate_all_metrics()

print("\nOptimized Path Metrics:")
for metric_name, value in optimized_metrics.items():
    print(f"{metric_name}: {value:.4f}")

# Visualize the optimized path
optimized_evaluator.visualize_paths(
    title="Optimized Path vs Reference Path",
    buffer_distance=100
)

## 9.2 Comparing Different Path-Finding Algorithms

Let's compare the A* algorithm with Dijkstra's algorithm using our optimized weights.

In [ ]:
# Compare A* and Dijkstra algorithms with optimized weights
print("\nComparing path-finding algorithms with optimized weights...")

# Generate cost map with optimized weights
comparison_cost_map = cost_generator.generate_cost_map(
    weights=best_weights,
    default_weight=0.05
)

# Create path finder
path_finder = LeastCostPathFinder(comparison_cost_map)

# Find paths using both algorithms
astar_start_time = time.time()
astar_path_indices, astar_path_cost = path_finder.find_path(
    start_indices,
    end_indices,
    method='astar'
)
astar_time = time.time() - astar_start_time

dijkstra_start_time = time.time()
dijkstra_path_indices, dijkstra_path_cost = path_finder.find_path(
    start_indices,
    end_indices,
    method='dijkstra'
)
dijkstra_time = time.time() - dijkstra_start_time

# Display results
print(f"A* algorithm results:")
print(f"  Path cost: {astar_path_cost:.4f}")
print(f"  Execution time: {astar_time:.4f} seconds")
print(f"  Path length: {len(astar_path_indices)} points")

print(f"\nDijkstra's algorithm results:")
print(f"  Path cost: {dijkstra_path_cost:.4f}")
print(f"  Execution time: {dijkstra_time:.4f} seconds")
print(f"  Path length: {len(dijkstra_path_indices)} points")

# Evaluate both paths
astar_evaluator = PathEvaluator(
    predicted_path=astar_path_indices,
    reference_path=reference_path,
    transform=metadata['transform'],
    crs=metadata['crs']
)

dijkstra_evaluator = PathEvaluator(
    predicted_path=dijkstra_path_indices,
    reference_path=reference_path,
    transform=metadata['transform'],
    crs=metadata['crs']
)

# Calculate metrics
astar_metrics = astar_evaluator.evaluate_all_metrics()
dijkstra_metrics = dijkstra_evaluator.evaluate_all_metrics()

print("\nA* metrics:")
for metric_name, value in astar_metrics.items():
    print(f"  {metric_name}: {value:.4f}")
    
print("\nDijkstra metrics:")
for metric_name, value in dijkstra_metrics.items():
    print(f"  {metric_name}: {value:.4f}")

# Visualize both paths together
plt.figure(figsize=(12, 10))
plt.imshow(comparison_cost_map, cmap='viridis', alpha=0.7)
plt.colorbar(label='Cost')

# Plot the paths
astar_rows, astar_cols = astar_path_indices[:, 0], astar_path_indices[:, 1]
dijkstra_rows, dijkstra_cols = dijkstra_path_indices[:, 0], dijkstra_path_indices[:, 1]

plt.plot(astar_cols, astar_rows, 'r-', linewidth=2, label='A* Path')
plt.plot(dijkstra_cols, dijkstra_rows, 'b--', linewidth=2, label='Dijkstra Path')
plt.plot(start_indices[1], start_indices[0], 'go', markersize=10, label='Start')
plt.plot(end_indices[1], end_indices[0], 'mo', markersize=10, label='End')

plt.title('Comparison of Path-Finding Algorithms')
plt.legend()
plt.xlabel('X')
plt.ylabel('Y')
plt.tight_layout()
plt.show()

## 10. Neural Network-Based Weight Prediction (Extension)

Now let's explore how to use a neural network to predict weights based on dataset features:

```python
from pipeline.weight_optimizer import NeuralWeightPredictor, extract_dataset_features
import numpy as np

# For demonstration purposes, let's assume we have multiple datasets
# and we've already found optimal weights for each

# Create example dataset for training
num_datasets = 5
feature_dim = 20  # Number of features per dataset

# Create dummy training data (in a real scenario, this would be extracted from real datasets)
X_train = np.random.rand(num_datasets, feature_dim)
y_train = np.random.rand(num_datasets, len(factor_names))
y_train = y_train / y_train.sum(axis=1, keepdims=True)  # Normalize weights to sum to 1

# Create and train the neural weight predictor
nn_predictor = NeuralWeightPredictor(input_shape=(feature_dim,))
nn_predictor.factor_names = factor_names  # Set factor names for the model

# Train the model
history = nn_predictor.train(X_train, y_train, epochs=30, batch_size=2)

# Now we can use the model to predict weights for a new dataset
# Extract features from our dataset
dataset_features = extract_dataset_features(data_loader)

# Predict weights
predicted_weights = nn_predictor.predict_weights(dataset_features)

print("\nNeural Network Predicted Weights:")
for factor, weight in predicted_weights.items():
    print(f"{factor}: {weight:.4f}")
```

## 11. Process Multiple Datasets

Let's use the enhanced `process_dataset` function to handle multiple datasets:

In [ ]:
def process_dataset(dataset_path, weights, start_coords=None, end_coords=None, factor_mapping=None):
    """Process a single dataset and return metrics."""
    # Load data
    data_loader = DataLoader(dataset_path)
    data_loader.load_data(factor_mapping=factor_mapping)
    
    # Load reference path
    reference_path_dir = os.path.join(dataset_path, 'reference')
    reference_path = load_reference_path(reference_path_dir)
    
    # Generate cost map
    cost_generator = CostMapGenerator(data_loader.factors)
    cost_map = cost_generator.generate_cost_map(
        weights=weights,
        default_weight=0.05
    )
    
    # Get metadata
    metadata = data_loader.get_common_metadata()
    
    # Determine start and end points
    if start_coords and end_coords:
        start_indices = data_loader.coordinates_to_indices(start_coords[0], start_coords[1])
        end_indices = data_loader.coordinates_to_indices(end_coords[0], end_coords[1])
    else:
        # Use first and last points of reference path as default
        ref_geom = reference_path.geometry.iloc[0]
        start_coords = (ref_geom.coords[0][0], ref_geom.coords[0][1])
        end_coords = (ref_geom.coords[-1][0], ref_geom.coords[-1][1])
        start_indices = data_loader.coordinates_to_indices(start_coords[0], start_coords[1])
        end_indices = data_loader.coordinates_to_indices(end_coords[0], end_coords[1])
    
    # Find path using A* algorithm
    path_finder = LeastCostPathFinder(cost_map)
    path_indices, path_cost = path_finder.find_path(
        start_indices, 
        end_indices, 
        method='astar'
    )
    
    # Evaluate path
    evaluator = PathEvaluator(
        predicted_path=path_indices,
        reference_path=reference_path,  # Pass reference path directly
        transform=metadata['transform'],
        crs=metadata['crs']
    )
    
    metrics = evaluator.evaluate_all_metrics()
    
    # Return results
    return {
        'dataset': os.path.basename(dataset_path),
        'cost_map': cost_map,
        'path_indices': path_indices,
        'path_cost': path_cost,
        'metrics': metrics,
        'data_loader': data_loader,
        'metadata': metadata,
        'reference_path': reference_path
    }

# Define factor and weight mappings (if needed)
factor_mappings = {
    'dataset1': {
        'elev.tif': 'elevation',
        'protected.tif': 'protected_areas',
        'cities.tif': 'proximity_to_cities',
        'slope.tif': 'slope',
        'soil.tif': 'soil_type'
    },
    'dataset2': {
        'dem.tif': 'elevation',
        'parks.tif': 'protected_areas',
        'urban.tif': 'proximity_to_cities'
        # Missing factors will use default weight
    }
}

# Process all datasets with the optimized weights
all_results = []
for dataset_path in dataset_paths:
    dataset_name = os.path.basename(dataset_path)
    dataset_factor_mapping = factor_mappings.get(dataset_name)
    
    result = process_dataset(
        dataset_path, 
        best_weights,
        factor_mapping=dataset_factor_mapping
    )
    all_results.append(result)
    
    # Print metrics
    print(f"\nDataset: {result['dataset']}")
    for metric_name, value in result['metrics'].items():
        print(f"{metric_name}: {value:.4f}")

## 12. Conclusion

In this notebook, we've demonstrated the enhanced pipeline suitability analysis framework:

1. We loaded raster factors with support for custom factor name mapping
2. We loaded reference paths separately from the data loader
3. We created cost maps with weighted factors, supporting default weights and factor mapping
4. We found least cost paths using two algorithms: A* (more efficient) and Dijkstra's algorithm
5. We compared predicted paths with reference paths using multiple metrics
6. We optimized weights to improve path prediction
7. We showed how to handle datasets with different factor names and structures
8. We compared performance and results between different path-finding algorithms

The framework now provides:
- More flexibility in handling different factor names and structures
- Support for multiple path-finding algorithms
- Clear separation of concerns between data loading and path evaluation
- Default path-finding behavior for demonstration purposes

This framework can be used as a foundation for further refinements, including:
- More sophisticated neural network architectures for weight prediction
- Adding more advanced path-finding algorithms
- Including additional path similarity metrics
- Scaling to larger datasets and more complex landscapes
- Interactive visualization and interactive weight adjustment